# Aplicando eigenfaces para reconhecimento facial

Feita a explicação teórica, vamos à aplicação prática. Nesse projeto, temos 20 imagens do artista William Smith e outras 20 da cantora Taylor Swift. Nosso objetivo é verificar se o método de eigenfaces é capaz de distingui-los e indentificá-los.  
Inicialmente, importamos as bibliotecas necessárias.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os

Para que o método funcione, é necessário que todas as imagens possuam as mesmas dimensões. Dessa forma, optamos por figuras de tamanho 195x231.

In [ ]:
altura_imagem = 231
comprimento = 195

A seguir, fizemos a leituras das imagens e as convertemos em uma matriz tridimensional (cada imagem é uma matriz bidimensional e sua composição adiciona uma terceira dimensão).

In [ ]:
# Criando o path para onde as imagens estão
famousim = "imagens_artistas/"
# Criando uma lista para adicionar as imagens
famous_images = []

# Fazendo um loop for para iterar sobre todas as imagens
for folder in (os.listdir(famousim)):
    folder_path = os.path.join(famousim, folder)
    for image in (os.listdir(folder_path)):
        # Adicionando o path com cada imagem para termos o arquivo das imagens
        path = os.path.join(famousim, folder ,image)
        # Lendo a imagem
        img = cv2.imread(path)
        # Convertendo a imagem para preto e branco 
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # Adicionando a imagem na lista
        famous_images.append(img)

# Transformando a lista num array
famous_images_np = np.array(famous_images)

Vamos dar uma olhada nas nossas imagens.

In [ ]:
# Plotando as imagens
count = 0
fig, ax = plt.subplots(5, 8, figsize=(10, 10))
for linha in range(5):
    for coluna in range(8):
        ax[linha,coluna].imshow(famous_images_np[count], cmap='gray')
        count += 1
        ax[linha, coluna].set_xticks([])
        ax[linha, coluna].set_yticks([])
        ax[linha, coluna].set_xticklabels([])
        ax[linha, coluna].set_yticklabels([])
        ax[linha, coluna].set_xlabel('')
        ax[linha, coluna].set_ylabel('')

Agora, transformamos essa matriz tridimensional em uma matriz bidimensional convertendo as matrizes de cada imagem para um único vetor-linha extremamente comprido. Após isso, calculamos a face média.

In [ ]:
# Cada imagem está armazenada em 3 dimensões, cada foto está numa lista, e cada lista é uma lista de listas.
# Vamos transformar a imagem num vetor coluna muito grande.
train_famous_np_matrix = famous_images_np.reshape(famous_images_np.shape[0], famous_images_np.shape[1]*famous_images_np.shape[2])
# Calculamos a face média
mean_train_face = np.mean(train_famous_np_matrix, axis=0)
# Criamos uma matriz subtraindo a face média
centered_train_face = train_famous_np_matrix - mean_train_face

# Mostrando a face média
plt.figure()
plt.imshow(mean_train_face.reshape(altura_imagem,comprimento), cmap='gray')


A seguir, calculamos a decomposição SVD dessa matriz.

In [ ]:
# Decompomdo a matriz em SVD
U_Face_Famous, D_Face_Famous, V_Face_Famous = np.linalg.svd(centered_train_face, full_matrices=False)

Vamos selecionar as 12 primeiras componentes principais e plotar suas eigenfaces.

In [ ]:
# Vamos obter a quantidade de componentes que queremos de nossa matriz.
C_Faces_TRAIN = V_Face_Famous[:12]
eigen_faces_train = C_Faces_TRAIN.reshape((-1, altura_imagem, comprimento))

In [ ]:
count = 0
fig, ax = plt.subplots(3, 4, figsize=(10, 10))
for linha in range(3):
    for coluna in range(4):
        ax[linha,coluna].imshow(eigen_faces_train[count], cmap='gray')
        count += 1
        ax[linha, coluna].set_xticks([])
        ax[linha, coluna].set_yticks([])
        ax[linha, coluna].set_xticklabels([])
        ax[linha, coluna].set_yticklabels([])
        ax[linha, coluna].set_xlabel('')
        ax[linha, coluna].set_ylabel('')

Vamos dar uma olhada em nossas imagens agora centralizadas (subtraídas a média).

In [ ]:
count = 0
fig, ax = plt.subplots(5, 8, figsize=(10, 10))
for linha in range(5):
    for coluna in range(8):
        ax[linha,coluna].imshow(centered_train_face[count].reshape(altura_imagem,comprimento), cmap='gray')
        count += 1
        ax[linha, coluna].set_xticks([])
        ax[linha, coluna].set_yticks([])
        ax[linha, coluna].set_xticklabels([])
        ax[linha, coluna].set_yticklabels([])
        ax[linha, coluna].set_xlabel('')
        ax[linha, coluna].set_ylabel('')

A seguir, vamos analisar a variância cumulativa das 40 componentes principais. Desse modo, podemos considerar apenas as mais relevantes a fim de simplificar a análise do dado.

In [ ]:
# O quanto cada autovalor representa do dado?

# Somando todos os autovalores
sum_all = sum(D_Face_Famous)
# Transformando o array com os autovalores numa soma cumulativa
cummulative = np.cumsum(D_Face_Famous).round(2)
# Vendo quanto representa do total
percentage = cummulative/sum_all

# Plotando o gráfico
plt.bar(range(len(percentage)), percentage, color='blue', alpha=0.7)
plt.title("Gráfico de variância por quantidade de PCA's")
plt.xlabel("Quantidade de PCA's")
plt.ylabel("% de explicação do dado")


Podemos observar que as 20 primeiras componentes, ou seja, metade do conjunto, é capaz de representar aproximadamente 70% do dado. No entanto, para fins lúdicos, utilizaremos a seguir apenas as 2 primeiras a fim de ser possível plotá-las em um gráfico. Apesar de não serem tão precisas, elas já representam 15% da variação do dado, sendo suficiente para nosso propósito.

In [ ]:

# Nossa ideia agora, será pegar os dois principais componentes encontradas pelo PCA e então plotar as nossas imagens nesse subespaço
dois_maiores_PCAs = V_Face_Famous[:2]

# Criando a lista o qual terão as coordenadas das imagens que já trainamos o nosso dado
coordinates = []

# Fazendo um loop for para adicionar todas as imagens nessa lista
for img in centered_train_face:
    # Fazendo o produto interno para achar a projeção de cada face no espaço das duas principais componentes.
    coordinates_xy = np.dot(dois_maiores_PCAs, img)
    # Adicionando na lista
    coordinates.append(coordinates_xy)


Portanto, chequemos a distribuição do dado a partir dessas duas componentes principais. No gráfico abaixo, os pontos vermelhos representam as imagens da Taylor Swift, enquanto que os azuis representam as do Will Smith.

In [ ]:
# Colocando uma lista de cores para diferenciar taylor swift e will smith
colors = []
iteracao = 1

for number in range(len(coordinates)):
    if iteracao <= 20:
        colors.append("red")
        iteracao += 1
    elif iteracao <= 40:
        colors.append("blue")
        iteracao += 1

coordinates = np.array(coordinates).T

x_line = np.linspace(-8000, 8000, 100)
y_line = -0.55*x_line - 700

plt.scatter(coordinates[0], coordinates[1], c=colors)
plt.plot(x_line, y_line, color = "black")
plt.show()

# Redefinindo as coordenadas para o próximo gráfico
coordinates = []
for img in centered_train_face:
    coordinates_xy = np.dot(dois_maiores_PCAs, img)
    coordinates.append(coordinates_xy)

Apesar da baixa acurácia fornecida apenas por 2 componentes, obtemos uma distribuição relativamente distinguível, com apenas alguns pontos em posições incorretas.  
Agora, vamos brincar um pouco e testar algumas imagens de fora da base para ver onde elas serão colocadas nessa distribuição. Testaremos outras imagens da própria Taylor Swift e do próprio Will Smith, além de outras 4 celebridades.  
Em cada um dos gráficos abaixo, o ponto preto representa a nova imagem. Lembrando que os pontos vermelhos representam a Taylor Swift e os azuis o Will Smith.

In [ ]:
# Agora que já temos os dados dos elementos que plotamos, vamos tentar pegar imagens que não foram utilizadas e então plotar no nosso espaço para determinar se a pessoa é mais parecida com o will smith ou com a talor swift.

# Adicionando o path
training_set = "imagens_datateste/"

# criando um dicionário que terá o nome das pessoas e as coordenadas
coordinates_new_dict = {}

# Iterando sobre cada imagem da pasta
for image_name in os.listdir(training_set):
    # Pegando o caminho até ela
    image_path = os.path.join(training_set, image_name)

    # Realizando procedimentos pra ler a imagem
    img = cv2.imread(image_path)
    # Transformando num cinza
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Transformando em array
    img = np.array(img)
    # transformando num array unidimensional
    img = img.flatten()
    # retirando a imagem nova
    img = img - mean_train_face

    # Fazendo o produto interno para achar a projeção no espaço que queremos
    coordinates_new = np.dot(dois_maiores_PCAs, img)

    # Colocando as coordenadas num dicionário
    coordinates_new_dict[str(image_name[0:-4])] = coordinates_new


# Colocando uma lista de cores para diferenciar taylor swift e will smith
colors = []
iteracao = 1

for number in range(len(coordinates)):
    if iteracao <= 20:
        colors.append("red")
        iteracao += 1
    elif iteracao <= 40:
        colors.append("blue")
        iteracao += 1

# Definindo a cor do teste
colors.append("black")


In [ ]:

# Criando um grid plot com as pessoas
fig, axes = plt.subplots(3, 2, figsize=(10, 8))
axes = axes.flatten()

# Criando um loop para iterar sobre cada plot
for i, (name, values) in enumerate(coordinates_new_dict.items()):
    ax = axes[i]
    
    ax.set_title(str(name)[1:])

    # criando uma cópia das coordenadas
    temp_coordinates = coordinates.copy()
    
    # colocando as coordenadas que estão no dicionário na lista
    temp_coordinates.append(values)
    
    # Transformando em array e transpondo a matriz
    temp_coordinates = np.array(temp_coordinates).T
    
    # Fazendo o scatter plot
    ax.scatter(temp_coordinates[0], temp_coordinates[1], c=colors)
    x_line = np.linspace(-8000, 8000, 100)
    y_line = -0.55*x_line - 700

    ax.plot(x_line, y_line, color = "black")

    axes[i].set_xticks([])
    axes[i].set_yticks([])
    axes[i].set_xticklabels([])
    axes[i].set_yticklabels([])
    axes[i].set_xlabel('PC1')
    axes[i].set_ylabel('PC2')

    

# Mostrando o gráfico
plt.tight_layout()
plt.show()    
    